## Developing a multiple feeds construction algorithm

### Imports

In [1]:
import scipy as sp
import scipy.integrate
import matplotlib.pyplot as plt
from scipy import linalg
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import ConvexHull
from __future__ import division

%matplotlib inline
plt.style.use('ggplot')

### `artools.py`

In [2]:
import sys
sys.path.append('../artools/')
import artools
artools = reload(artools)

### Define `stoich_subspace()`

In [3]:
def stoich_subspace(Cf0s, stoich_mat):
    """ 
    Compute the bounds of the stoichiometric subspace, S, from multiple feed points and a stoichoimetric coefficient matrix.

    Parameters:
    
        stoich_mat    (n x d) array. Each row in stoich_mat corresponds to a component and each column corresponds to a reaction.
        
        Cf0s          (M x n) matrix. Each row in Cf0s corresponds to an individual feed and each column corresponds to a component.


    Returns:
    
        S_attributes   dictionary that contains the vertices stoichiometric subspace in extent and concentration space for individual feeds                        as well as overall stoichiometric subspace for multiple feeds.                         
        
        keys:
        
            all_Es      vertices of the individual stoichiometric subspaces in extent space.

            all_Cs      vertices of the individual stoichiometric subspaces in concentration space.

            all_Es_mat  list of vertices of the overall stoichiometric subspace in extent space.

            all_Cs_mat  list of vertices of the overall stoichiometric subspace in concentration space.

            hull_Es     extreme vertices of the overall stoichiometric subspace in the extent space.              

            hull_Cs     extreme vertices of the overall stoichiometric subspace in concentration space.

            bounds      bounds of the stoichiometric subspace in concentration space.

    """
    
    # create an empty list of bounds/ axis_lims
    min_lims = []
    max_lims = []
    
    # to store stoichSubspace_attributes
    S_attributes = {}
    
    # to store vertices for each feed and stoich_mat in extent and concentration space  
    all_Es = []
    all_Cs = []
    
    # if user input is not a list, then convert into a list 
    if not isinstance(Cf0s, list) and not Cf0s.shape[0] > 1 and not Cf0s.shape[1] > 1:
        # put it in a list 
        Cf0s = [Cf0s]
    
    for Cf0 in Cf0s:
        # loop through each feed point, Cf0, and check if it is a column vector 
        # with ndim=2, or a (L,) array with ndim=1 only
        if Cf0.ndim == 2:
            Cf0 = Cf0.flatten() # converts into (L,)
            
        # raise an error if the no. of components is inconsistent between the feed and stoichiometric matrix
        if len(Cf0) != stoich_mat.shape[0]:
            raise Exception("The number of components in the feed does not match the number of rows in the stoichiometric matrix.")
            
        # always treat stoich_mat as a matrix for consistency, convert if not
        if stoich_mat.ndim == 1: 
            # converts a 'single rxn' row into column vector  
            stoich_mat = stoich_mat.reshape((len(stoich_mat), 1)) 

        # check if  a single reaction or multiple reactions are occuring  
        if stoich_mat.shape[1] == 1 or stoich_mat.ndim == 1: 
            # if stoich_mat is (L,) array this'stoich_mat.shape[1]' raises an error 'tuple out of range'  
            
            # converts into (L,)
            stoich_mat = stoich_mat.flatten()

            # calculate the limiting requirements
            limiting = Cf0/ stoich_mat

            # only choose negative coefficients as these indicate reactants
            k = limiting < 0.0

            # calc maximum extent based on limiting reactant and calc C
            # we take max() because of the negative convention of the limiting requirements 
            e_max = sp.fabs(max(limiting[k]))
            
            # calc the corresponding point in concentration space 
            C = Cf0 + stoich_mat*e_max

            # form Cs and Es and return
            Cs = sp.vstack([Cf0, C])
            Es = sp.array([[0., e_max]]).T

        else:
            # extent associated with each feed vector
            Es = artools.con2vert(- stoich_mat, Cf0) 
            
            # calc the corresponding point in concentration space
            Cs = (Cf0[:, None] + sp.dot(stoich_mat, Es.T)).T 

        # vertices for each feed and stoich_mat in extent and concentration space
        all_Es.append(Es) 
        all_Cs.append(Cs)

        # stack vertices in one list and find the overall stoichiometric subspace(convex hull) 
        all_Es_mat = sp.vstack(all_Es)
        all_Cs_mat = sp.vstack(all_Cs)
    
    # compute the convexhull of the overall stoichiometric subspace 
    # if n > d + 1, then hull_Cs is returned as the full list of vertices 
    if len(Cf0) > artools.rank(stoich_mat) + 1:
        # convexHull vertices are returned as the whole stack of points
        hull_Es = all_Es_mat
        hull_Cs = all_Cs_mat
    else:
        # convexHull vertices for the overall stoichiometric subspace in extent space         
        hull_all = ConvexHull(all_Es_mat)
        ks = hull_all.vertices
        hull_Es = all_Es_mat[ks, :]

        # convexHull vertices for the overall stoichiometric subspace in concentration space
        hull_all = ConvexHull(all_Cs_mat)
        ks = hull_all.vertices
        hull_Cs = all_Cs_mat[ks, :] 
    
    # no. of components
    N = stoich_mat.shape[0]

    # create a matrix of indices 
    components = sp.linspace(0, N-1, num=N)  
    
    for i in components:
        # loop through each component and find the (min, max) => bounds of the axis  
        minMatrix = min(hull_Cs[:, i])
        maxMatrix = max(hull_Cs[:, i])

        # append limits into preallocated lists (min_lims, max_lims)
        min_lims.append(minMatrix)
        max_lims.append(maxMatrix)

        # stack them into an ndarray and flatten() into a row vector 
        bounds = sp.vstack((min_lims, max_lims)).T
        bounds = bounds.flatten() # alternating min, max values

    # create a dictionary containing all the 'attributes' of the 'stoich_subspace'
    S_attributes = {
        'all_Es' : all_Es,
        'all_Cs' : all_Cs,
        'all_Es_mat' : all_Es_mat,
        'all_Cs_mat' : all_Cs_mat,
        'hull_Es' : hull_Es,
        'hull_Cs' : hull_Cs,
        'bounds' : bounds
}
        
    return S_attributes

### Supply stoichiometric coefficient and multiple feeds test data

In [4]:
stoich_mat = sp.array([[-1, 1, 0, 0], [0, -1, 1, 0], [-2, 0, 0, 1]]).T

Cf1 = sp.array([1., 0, 0, 0])
Cf2 = sp.array([1.5, 0.1, 0, 0])
Cf3 = sp.array([0.5, 0.5, 0.5, 0.5])
Cf0s = sp.array(([Cf1, Cf2, Cf3]))

### Find the stoichiometric subspace 

In [5]:
S_attributes = stoich_subspace(Cf0s, stoich_mat)
Cs = S_attributes['hull_Cs']

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:129: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:130: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


### define test rate kinetics (van der Vusse system) 

In [6]:
def rate_func(C, t):
    """determine rate vector from a given set of rate kinetics"""
    
    # declare concentration variables
    cA = C[0];
    cB = C[1];
    cC = C[2];
    cD = C[3];
    
    # rate constants  
    k1 = 1.0;               
    k2 = 1.0; 
    k3 = 10.0;
    
    # define rate vector
    # rate vector = [rA, rB, rC, rD]  (mol/L.s)
    R = sp.array([-k1*cA - 2*k3*cA**2,
                    k1*cA - k2*cB,
                    k2*cB,
                    k3*cA**2]);
    return R

### define PFR integration time 

In [7]:
pfr_ts = sp.logspace(sp.log10(1e-5), sp.log10(10), num=50)

### loop through each feed and find PFR trajectory

In [8]:
# find individual PFR trajectories
pfr_cs_1 = scipy.integrate.odeint(rate_func, Cf1, pfr_ts)
pfr_cs_2 = scipy.integrate.odeint(rate_func, Cf2, pfr_ts)
pfr_cs_3 = scipy.integrate.odeint(rate_func, Cf3, pfr_ts)

# find convexHull of PFRs originating from multiple feeds
pts = sp.vstack((pfr_cs_1, pfr_cs_2, pfr_cs_3))
all_pts = ConvexHull(pts)
ks = all_pts.vertices
hull_pts = pts[ks, :]

### Plot stoichiometric subspace from given reaction stoichiometry and multiple feed points

In [9]:
# slicing Cs matrix => (n = d), to avoid dimensional error in the `plot_region3d` function 
Cs = Cs[:, 0:3]

% matplotlib qt
fig1 = artools.plot_region3d(Cs, color="b", alpha=0.25)
fig1.hold(True)
ax1 = fig1.gca()

# to avoid dimensional error 
hull_pts = hull_pts[:, 0:3]

# create a new figure in the same axis 
fig2 = artools.plot_region3d(hull_pts, ax=ax1)

# plot feed points
ax1.plot(Cf0s[:, 0], Cf0s[:, 1], Cf0s[:, 2], 'bo')

# plot PFRs from feed points
ax1.plot(pfr_cs_1[:, 0], pfr_cs_1[:, 1], pfr_cs_1[:, 2], 'r')
ax1.plot(pfr_cs_2[:, 0], pfr_cs_2[:, 1], pfr_cs_2[:, 2], 'r')
ax1.plot(pfr_cs_3[:, 0], pfr_cs_3[:, 1], pfr_cs_3[:, 2], 'r')

# plot points
ax1.plot( Cs[:, 0], Cs[:, 1], Cs[:, 2], 'k.')

ax1.set_xlabel('cA (mol/ L)')
ax1.set_ylabel('cB (mol/ L)')
ax1.set_zlabel('cC (mol/ L)')
ax1.set_title('Stoichiometric subspace for a multiple feed van der Vusse system')

# plt.show(fig1)    

C:\Anaconda2\lib\site-packages\matplotlib\collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


### Search for achievable points in the complement space 

- find stoichiometric subspace
- find initial candidate region composed of PFRs originating from multiple feed points
- define complement region region
- spit out random points in complement space and check for achievability
    - the achievability condition is checking if the rate vector is collinear to the vector (C - C*)
    - if the achievability condition is met, then the point is achievable and it is added onto the list of achievable points, and subsequently, the region is expanded
    - if not, then the point is unachievable in the current iteration
    

### Spit out random points
- using `rand_pts` function in ARtools

In [10]:
# define axis limits
axis_lims = S_attributes['bounds']

# define no. of points
Npts = 10000

# generate random points
C = artools.rand_pts(Npts, axis_lims)

### Generate random points within feasible region
- employ facet enumeration to find inequality equations for the stoichiometric subspace
    - create another attribute for hyperplane inequality equations

In [11]:
#find hyperplane inequality equations using vert2con() (facet enumeration)
A, b = artools.vert2con(Cs)

### Use `in_region` to find points with the stoichiometric subspace

In [12]:
ks = []
for i, ci in enumerate(C):
    if artools.in_region(ci, A, b):
        ks.append(i)

ValueError: shapes (4,3) and (4,) not aligned: 3 (dim 1) != 4 (dim 0)

### Plot 'possible' points

In [ ]:
# plot random points in the feasible region
% matplotlib qt
fig1 = artools.plot_region3d(Cs, color="b", alpha=0.25)
fig1.hold(True)
ax1 = fig1.gca()

# to avoid dimensional error 
hull_pts = hull_pts[:, 0:3]

# create a new figure in the same axis 
fig2 = artools.plot_region3d(hull_pts, ax=ax1)

# plot feed points
ax1.plot(Cf0s[:, 0], Cf0s[:, 1], Cf0s[:, 2], 'bo')

C = C[:, 0:3]
ax1.plot(C[ks, 0], C[ks, 1], C[ks, 2], 'r.')

# plot PFRs from feed points
ax1.plot(pfr_cs_1[:, 0], pfr_cs_1[:, 1], pfr_cs_1[:, 2], 'r')
ax1.plot(pfr_cs_2[:, 0], pfr_cs_2[:, 1], pfr_cs_2[:, 2], 'r')
ax1.plot(pfr_cs_3[:, 0], pfr_cs_3[:, 1], pfr_cs_3[:, 2], 'r')

# plot points
ax1.plot( Cs[:, 0], Cs[:, 1], Cs[:, 2], 'k.')

ax1.set_xlabel('cA (mol/ L)')
ax1.set_ylabel('cB (mol/ L)')
ax1.set_zlabel('cC (mol/ L)')
ax1.set_title('random points in stoich_subspace')

plt.show(fig1) 